In [2]:
from binance.client import Client
from binance.enums import *
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [ ]:
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

def getminutedata(symbol, interval, start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                        interval,
                                                        start + ' min ago UTC'))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

In [ ]:
value = getminutedata(artifact_table.iloc[artifact_id]['ticker'], 
                                    '15m',
                                    '120')
close = value[['Time','Close']]                             
close.index = pd.to_datetime(close['Time'])
close = close.drop(columns=['Time'])
close = close.rename(columns={"Close": f"{artifact_id}"})

In [3]:
from sqlalchemy import create_engine
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)

In [9]:
trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
trend_table_rowcount = trend_table_status['Rows'].values[0]
trend_table_rowcount

In [24]:
check_date = pd.read_sql_query("SELECT MAX(date_request) FROM crypto_trend", engine) 
date_recent_checked = str(check_date.iloc[0, 0])

In [25]:
date_recent_checked

'2022-03-02 13:13:00'

In [ ]:
maxholds = 3
trades = [[],[],[],[]]
tradedata = []
buyPrice  = [0] * (len(id_list))
sellPrice = [0] * (len(id_list))
tradeBuy  = [0] * (len(id_list))
tradeSell = [0] * (len(id_list))
buyTime   = [0] * (len(id_list))
sellTime  = [0] * (len(id_list))
holding = [False] * (len(id_list))
buy_threshold = 80
sell_threshold = 50

In [ ]:
import time
run = 1
while (run == 1):
    ##############             Bot Program             ##############
    # Fetch ID close prices  & trend 
    trends, closes = fetch_data(id_list, artifact_table, crypto_table)
    
    # Fetch ID's for buy & sell indicator
    indicate_buy, indicate_sell = fetch_signals(buy_threshold, sell_threshold, trends, closes)

    #upload indicate buy and trends and closes to database


    buy = np.logical_and( indicate_buy, np.logical_not(holding) )
    buy_ids = [id_list[i] for i in [i for i, x in enumerate(buy) if x]]
    if sum(buy) > 0:
        print([trends.index[-1], buy_ids])
    #print(buy_ids)
    sell = np.logical_and(indicate_sell, holding)
    sell_ids = [id_list[i] for i in [i for i, x in enumerate(sell) if x]]
    #if sum(sell) > 0:
    #    print([trends.index[-1], sell_ids])
    #print(sell_ids)

    # Buy control
    if ((sum(holding) < maxholds) and buy_ids):
        for id in reversed(buy_ids):
            if (sum(holding) < maxholds):
                key = id_list.index(id)
                tradetime = closes.index[-1]
                buyPrice[key] = closes.loc[tradetime,str(id)]
                buyTime[key] = tradetime
                holding[key] = True
                # Buy execute function binance using ticker
            else:
                continue
    # Sell control
    if (sell_ids):
        for id in sell_ids:
            # Sell execute function binance using ticker
            key = id_list.index(id)
            tradetime = closes.index[-1]
            sellPrice[key] = closes.loc[tradetime,str(id)]
            sellTime[key] = tradetime
            tradedata.append([id, buyTime[key], sellTime[key], buyPrice[key], sellPrice[key], (sellPrice[key]/buyPrice[key]) ])
            holding[key] = False
    print("Finished cycle " + str(datetime.datetime.now()))
    
    run = 0
    time.sleep(60*30*2)
    # Calculate profit
    # tradedata = np.array(tradedata)
    # scaled = ((tradedata[:,5] - 1) * 1/maxholds) + 1
    # scaled.prod()
